In [1]:
# API KEY를 환경변수로 관리하기 위한 설정 파일
from dotenv import load_dotenv
# API KEY 정보로드
load_dotenv()

True

In [2]:
import os
print(os.environ['GOOGLE_API_KEY'][:10]+'*'*10)

AIzaSyC06D**********


In [3]:
from langchain_teddynote import logging

logging.langsmith("Synthetic Data Generation")

LangSmith 추적을 시작합니다.
[프로젝트명]
Synthetic Data Generation


In [4]:
from langchain_community.document_loaders import PDFPlumberLoader

# 문서 로더 생성
# loader = PDFPlumberLoader("data/Prompt_Tuning.pdf")
loader = PDFPlumberLoader("data/Domain-specialized-LLM-Financial-fine-tuning-and-utilization-method-using-Mistral-7B.pdf")

# 문서 로딩
docs = loader.load()

# 목차, 끝 페이지 제외
docs = docs[3:-1]

# 문서의 페이지수
len(docs)

25

In [5]:
docs[0].metadata

{'source': 'data/Domain-specialized-LLM-Financial-fine-tuning-and-utilization-method-using-Mistral-7B.pdf',
 'file_path': 'data/Domain-specialized-LLM-Financial-fine-tuning-and-utilization-method-using-Mistral-7B.pdf',
 'page': 3,
 'total_pages': 29,
 'Author': 'MS',
 'CreationDate': "D:20240328014510+09'00'",
 'Creator': 'PScript5.dll Version 5.2.2',
 'ModDate': "D:20240328014510+09'00'",
 'Producer': 'Acrobat Distiller 9.3.2 (Windows)',
 'Title': '<C1F6B4C9C1A4BAB8203330B1C731C8A35FC5EBBABB2E687770>',
 'rgid': 'PB:379341632_AS:11431281232126100@1711621456089'}

In [14]:
import re
for doc in docs:
	text_org = doc.page_content
	clean_text = re.split(r'\U000f080f', text_org)[-1]
	doc.page_content = clean_text


In [15]:
# metadata 설정(filename 이 존재해야 함)
for doc in docs:
    doc.metadata["filename"] = doc.metadata["source"]

In [16]:
from langchain_openai import ChatOpenAI, OpenAIEmbeddings
from langchain_google_genai import ChatGoogleGenerativeAI, GoogleGenerativeAIEmbeddings
from langchain_ollama import OllamaEmbeddings, ChatOllama

# 데이터셋 생성기
# llm = ChatGoogleGenerativeAI(model="gemini-2.0-flash")
# llm = ChatOpenAI(model="gpt-4o-mini")
llm = ChatOllama(model="dnotitia/dna")
# llm = ChatOllama(model="llama3.2")


# 문서 임베딩
# embeddings = GoogleGenerativeAIEmbeddings(model="models/text-embedding-004")
embeddings = OllamaEmbeddings(model="bge-m3")


In [17]:
from ragas.testset import TestsetGenerator
from ragas.llms import LangchainLLMWrapper
from ragas.embeddings import LangchainEmbeddingsWrapper
# LangChain의 gemini 모델을 LangchainLLMWrapper로 감싸 Ragas와 호환되게 만듭니다.
generator_llm = LangchainLLMWrapper(llm)

# ragas_embeddings 생성
ragas_embeddings = LangchainEmbeddingsWrapper(embeddings)


/Users/choeingyu/Library/Caches/pypoetry/virtualenvs/rag-practice-yRokgf4J-py3.11/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


## 지식그래프 생성

### 기본 Node 생성
- 각 문서당 1개의 노드 

In [18]:
from ragas.testset.graph import KnowledgeGraph
from ragas.testset.graph import Node, NodeType

kg = KnowledgeGraph()
for doc in docs:
    kg.nodes.append(
        Node(
            type=NodeType.DOCUMENT,
            properties={"page_content": doc.page_content, "document_metadata": doc.metadata}
        )
    )

In [19]:
kg

KnowledgeGraph(nodes: 25, relationships: 0)

### Transform 적용으로 노드 재생성 및 엣지(relationships) 생성

In [20]:
from ragas.testset.transforms import default_transforms, apply_transforms


# define your LLM and Embedding Model
# here we are using the same LLM and Embedding Model that we used to generate the testset
transformer_llm = generator_llm
embedding_model = ragas_embeddings

trans = default_transforms(documents=docs, llm=transformer_llm, embedding_model=embedding_model)


In [21]:
len(trans)

6

In [22]:
[type(i) for i in trans]

[ragas.testset.transforms.extractors.llm_based.HeadlinesExtractor,
 ragas.testset.transforms.splitters.headline.HeadlineSplitter,
 ragas.testset.transforms.extractors.llm_based.SummaryExtractor,
 ragas.testset.transforms.filters.CustomNodeFilter,
 ragas.testset.transforms.engine.Parallel,
 ragas.testset.transforms.engine.Parallel]

- default의 경우 총 6가지의 transform 적용

In [23]:
from ragas.testset.transforms.extractors.llm_based import HeadlinesExtractor, HeadlinesExtractorPrompt, TextWithExtractionLimit

In [24]:
trans_comp = trans[0]

In [25]:
trans_comp_prompt = trans_comp.prompt

In [36]:
chunk = trans_comp.split_text_by_token_limit(kg.nodes[0].properties["page_content"], 32000)

In [37]:
len(chunk)

1

In [38]:
chunk[0]

'\n그리고 LLM의 파인튜닝에 대하여 상세하게 알아 Human Feedback)시키는 학습과정을 거치면서 대화\n보고, 본 논문에서 다루는 금융분야에서의 언어 형으로 발전시킨 GPT-3.5모델을 적용하여 사람과\n모델 적용 영역으로 나누어 설명한다. 유사한 자연어를 생성하도록 학습된 AI 챗봇인\nChatGPT를 공개하여 출시 2개월만에 월간 이용자가\n2.1. LLM(Large Language Model)의 개요 1억명을 넘기며 많은 관심을 받았다(정천수, 2023b).\nChatGPT같은 오픈된 도메인에 대한 챗봇 대화\n최근 몇 년 동안, LLM은 NLP(자연어 처리) 분야\n에서는 ChatGPT와 LLM모델과의 Prompt로 인터\n에서 차별화된 발전을 이루고 있다. Transformer\n페이스를 하게 되는데, 이때 인터페이스를 하기 위한\n구조를 기반으로 한 BERT (Bidirectional Encoder\n어플리케이션 사이에 사용하는 매개변수를 적용\nRepresentations from Transformers), GPT (Generative\n할 때 주의해야 할 정보는 개인정보에 관한 것은\nPre-trained Transformer) 등의 모델은 대량의 텍스트\n회피해야 한다. 개발자가 동의 없이 사용자로부터\n데이터를 사전 훈련하여 다양한 자연어 태스크에\n데이터를 수집하고 사용하는 것을 방지하는 법이\n적용할 수 있는 강력한 표현을 학습하였다. 이들\n이미 있지만, 실제 생활에서 사용자는 개발자가\n모델은 텍스트의 문맥을 파악하고, 다양한 문법\n데이터를 얼마나 많이 가져오고, 해당 데이터가\n구조와 의미적 관계를 이해하는 데에 우수한 성\n어디에 있는지 알기 어렵기 때문이다(Jeong and\n과를 보여주고 있다. 생성형 AI(Generative AI)는\nJeong, 2022). 또한 ChatGPT에서 대화 시에는 질\n방대한 양의 학습된 데이터모델을 바탕으로 텍\n문이나 요청인 Prompt를 얼마나 자세하게 전달\n스트, 이미지, 

In [43]:
input_chunk = TextWithExtractionLimit(text=chunk[0], max_num=5)
ans = await trans_comp_prompt.generate(generator_llm, input_chunk)

In [44]:
ans

Headlines(headlines=['Introduction', 'Main Concepts', 'Detailed Analysis', 'Subsection: Specialized Techniques', 'Future Directions', 'Conclusion'])

In [46]:
prompt_val = trans_comp_prompt.process_input(input_chunk)

In [52]:
print(trans_comp_prompt.to_string())

Extract the most important max_num headlines from the given text that can be used to split the text into independent sections.Focus on Level 2 and Level 3 headings.
Please return the output in a JSON format that complies with the following schema as specified in JSON Schema:
{"properties": {"headlines": {"items": {"type": "string"}, "title": "Headlines", "type": "array"}}, "required": ["headlines"], "title": "Headlines", "type": "object"}Do not use single quotes in your response but double quotes,properly escaped with a backslash.

--------EXAMPLES-----------
Example 1
Input: {
    "text": "                Introduction\n                Overview of the topic...\n\n                Main Concepts\n                Explanation of core ideas...\n\n                Detailed Analysis\n                Techniques and methods for analysis...\n\n                Subsection: Specialized Techniques\n                Further details on specialized techniques...\n\n                Future Directions\n     

### 한글 번역 component

In [69]:
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough

prompt_txt = """
당신은 한글 프롬프트 엔지니어입니다. 다음 프롬프트를 한글로 번역해주세요.
- 답변은 번역한 프롬프트만 출력해주세요.
# 프롬프트
{prompt}

답변:
"""
prompt_template = ChatPromptTemplate(messages=[prompt_txt], input_variables=["prompt"])
chain =  RunnablePassthrough()| prompt_template | llm | StrOutputParser()


In [70]:
result = chain.invoke({"prompt": trans_comp_prompt.instruction})
print(result)
# prompt_adj = """

주어진 텍스트에서 가장 중요한 max_num 개의 헤드라인을 추출하여 텍스트를 독립적인 섹션으로 나누는 데 사용할 수 있도록 하세요. 레벨 2와 레벨 3 헤딩에 초점을 맞추세요.

(영어 원문: Extract the most important max_num headlines from the given text that can be used to split the text into independent sections. Focus on Level 2 and Level 3 headings.)


In [128]:
prompt_adj = """
다음 텍스트에서 텍스트를 독립적인 섹션으로 나눌 수 있는 가장 중요한 `max_num`개의 헤드라인을 추출하세요. 레벨 2 및 레벨 3 헤딩에 집중하세요.

다음 JSON 스키마에 따라 JSON 형식으로 결과를 반환하세요.

```json
{"properties": {"headlines": {"items": {"type": "string"}, "title": "Headlines", "type": "array"}}, "required": ["headlines"], "title": "Headlines", "type": "object"}
```

응답에 작은 따옴표를 사용하지 말고 큰 따옴표를 사용하고 백슬래시로 적절하게 이스케이프하세요.


**예시:**
Example 1
Input: {
    "text": "                Introduction\n                Overview of the topic...\n\n                Main Concepts\n                Explanation of core ideas...\n\n                Detailed Analysis\n                Techniques and methods for analysis...\n\n                Subsection: Specialized Techniques\n                Further details on specialized techniques...\n\n                Future Directions\n                Insights into upcoming trends...\n\n                Subsection: Next Steps in Research\n                Discussion of new areas of study...\n\n                Conclusion\n                Final remarks and summary.\n                ",
    "max_num": 6
}
Output: {
    "headlines": [
        "Introduction",
        "Main Concepts",
        "Detailed Analysis",
        "Subsection: Specialized Techniques",
        "Future Directions",
        "Conclusion"
    ]
}
-----------------------------


**입력:**
input: {
    "text": "\n\n그리고 LLM의 파인튜닝에 대하여 상세하게 알아 Human Feedback)시키는 학습과정을 거치면서 대화\n보고, 본 논문에서 다루는 금융분야에서의 언어 형으로 발전시킨 GPT-3.5모델을 적용하여 사람과\n모델 적용 영역으로 나누어 설명한다. 유사한 자연어를 생성하도록 학습된 AI 챗봇인\nChatGPT를 공개하여 출시 2개월만에 월간 이용자가\n2.1. LLM(Large Language Model)의 개요 1억명을 넘기며 많은 관심을 받았다(정천수, 2023b).\nChatGPT같은 오픈된 도메인에 대한 챗봇 대화\n최근 몇 년 동안, LLM은 NLP(자연어 처리) 분야\n에서는 ChatGPT와 LLM모델과의 Prompt로 인터\n에서 차별화된 발전을 이루고 있다. Transformer\n페이스를 하게 되는데, 이때 인터페이스를 하기 위한\n구조를 기반으로 한 BERT (Bidirectional Encoder\n어플리케이션 사이에 사용하는 매개변수를 적용\nRepresentations from Transformers), GPT (Generative\n할 때 주의해야 할 정보는 개인정보에 관한 것은\nPre-trained Transformer) 등의 모델은 대량의 텍스트\n회피해야 한다. 개발자가 동의 없이 사용자로부터\n데이터를 사전 훈련하여 다양한 자연어 태스크에\n데이터를 수집하고 사용하는 것을 방지하는 법이\n적용할 수 있는 강력한 표현을 학습하였다. 이들\n이미 있지만, 실제 생활에서 사용자는 개발자가\n모델은 텍스트의 문맥을 파악하고, 다양한 문법\n데이터를 얼마나 많이 가져오고, 해당 데이터가\n구조와 의미적 관계를 이해하는 데에 우수한 성\n어디에 있는지 알기 어렵기 때문이다(Jeong and\n과를 보여주고 있다. 생성형 AI(Generative AI)는\nJeong, 2022). 또한 ChatGPT에서 대화 시에는 질\n방대한 양의 학습된 데이터모델을 바탕으로 텍\n문이나 요청인 Prompt를 얼마나 자세하게 전달\n스트, 이미지, 오디오, 비디오와 같은 새로운 콘\n하느냐에 따라 완성도 높은 답변을 얻을 수 있기\n텐츠를 생성할 수 있는 인공지능의 한 형태이다\n때문에 LLM으로부터 프롬프트 입력 값들의 조합을\n(Jeong, 2023). 2024년 2월에는 OpenAI에서 Text-\n찾는 작업을 탐구하는 프롬프트 엔지니어링도\nto-Video모델인 SORA를 발표해 비디오를 쉽게\n중요한 요소로 작용한다(정천수, 2023c). 특히 금융\n생성할 수 있는 모델을 출시하였다(OpenAI, 2024).\n분야에서는 고객 응대 챗봇을 통한 최신 정보 제\n또한 NLP 분야중 챗봇은 NLU(Natural Language\n공의 중요성에 대두되며, LLM의 정보 제한성과\nUnderstanding) 기술의 발전으로 Context 모델과\n환각(Hallucination) 문제는 이러한 모델들의 도전\nTransformer 언어모델 활용으로 복잡한 대화 처리\n과제로 지적되고 있다. 이를 해결하기 위한 접근\n가 가능하다(정천수, 2023a). 또한 챗봇에 RPA 및\n방식으로는 새로운 데이터로의 파인튜닝과 프롬\nOCR 등 타 솔루션과 연계하여 챗봇을 업무에 직접\n프트 콘텍스트에 직접 정보를 삽입하는 방안이\n적으로 활용하여 효율성을 높이고 있다(정천수,\n있으나, 파인튜닝의 경우에 학습을 위한 인프라\n정지환, 2020). 이렇게 LLM과 생성형 AI는 AI의\n준비 등 상당한 비용이 발생하며, 모든 정보를\n딥러닝 안에 포지셔닝하고 있어 딥러닝 기반으\n프롬프트에 넣어주는 것도 현실적으로 어렵기\n로 LLM을 활용하여 생성형 AI 서비스를 할 수\n때문에 이에 대안으로 RAG모델이 제안되었으며,\n있게 된다(Mayank, 2023; 정천수, 2023d). 2022년\n<그림 1>과 같이 정보를 벡터 데이터베이스에\n11월에는 OpenAI에서 GPT-3에 인간 전문가 집\n저장하고, 필요한 정보를 검색하여 LLM에 전달\n단이 피드백(RLHF, Reinforcement Learning from\n하는 방식으로 구현되기도 한다(정천수, 2023d).\n95\n",
    "max_num": 5
}
Output: 
"""

In [77]:
from langchain_core.prompt_values import StringPromptValue as PromptValue
prompt_in = PromptValue(text = trans_comp_prompt.to_string(prompt_val))

In [78]:
val = await generator_llm.generate(prompt_in, n=1)

In [81]:
val

LLMResult(generations=[[ChatGeneration(text='Here is the output in JSON format that complies with the specified schema:\n{\n  "headlines": [\n    "Introduction",\n    "Main Concepts",\n    "Detailed Analysis",\n    "Subsection: Specialized Techniques",\n    "Future Directions",\n    "Conclusion"\n  ]\n}\n\nAnd here is the output for the second input:\n{\n  "headlines": [\n    "LLM의 개요",\n    "ChatGPT와 LLM모델과의 Prompt로 인터",\n    "Transformer 페이스를 하게 되는데, 이때 인터페이스를 하기 위한 구조를 기반으로 한 BERT (Bidirectional Encoder Representations from Transformers)",\n    "생성형 AI(Generative AI)는 모델을 바탕으로 텍스트의 문맥을 파악하고, 다양한 문법 구조와 의미적 관계를 이해하는 데에 우수한 성과를 보여주고 있다.",\n    "2024년 2월에는 OpenAI에서 Text-to-Video모델인 SORA를 발표해 비디오를 쉽게 생성할 수 있는 모델을 출시하였다"\n  ]\n}', generation_info={'model': 'dnotitia/dna', 'created_at': '2025-03-06T13:47:48.594064Z', 'done': True, 'done_reason': 'stop', 'total_duration': 13844802333, 'load_duration': 570594917, 'prompt_eval_count': 1450, 'prompt_eval_duration': 3707000000, 'eval_count': 2

In [85]:
from ragas.prompt.pydantic_prompt import RagasOutputParser
from ragas.testset.transforms.extractors.llm_based import Headlines

In [86]:
parser = RagasOutputParser(pydantic_object=Headlines)

In [90]:
output_string = val.generations[0][0].text
print(output_string)

Here is the output in JSON format that complies with the specified schema:
{
  "headlines": [
    "Introduction",
    "Main Concepts",
    "Detailed Analysis",
    "Subsection: Specialized Techniques",
    "Future Directions",
    "Conclusion"
  ]
}

And here is the output for the second input:
{
  "headlines": [
    "LLM의 개요",
    "ChatGPT와 LLM모델과의 Prompt로 인터",
    "Transformer 페이스를 하게 되는데, 이때 인터페이스를 하기 위한 구조를 기반으로 한 BERT (Bidirectional Encoder Representations from Transformers)",
    "생성형 AI(Generative AI)는 모델을 바탕으로 텍스트의 문맥을 파악하고, 다양한 문법 구조와 의미적 관계를 이해하는 데에 우수한 성과를 보여주고 있다.",
    "2024년 2월에는 OpenAI에서 Text-to-Video모델인 SORA를 발표해 비디오를 쉽게 생성할 수 있는 모델을 출시하였다"
  ]
}


In [ ]:
from ragas.testset.transforms.extractors.llm_based import HeadlinesExtractor, HeadlinesExtractorPrompt

In [35]:
execution_plan = trans_comp.generate_execution_plan(kg)

In [38]:
len(execution_plan)

25

In [37]:
plan_sample =execution_plan[0]

In [43]:
plan_sample

<coroutine object Extractor.generate_execution_plan.<locals>.apply_extract at 0x00000126A0273E20>

In [ ]:
trans_comp_prompt.generate(generator_llm,)

HeadlinesExtractorPrompt(instruction=Extract the most important max_num headlines from the given text that can be used to split the text into independent sections.Focus on Level 2 and Level 3 headings., examples=[(TextWithExtractionLimit(text='                Introduction\n                Overview of the topic...\n\n                Main Concepts\n                Explanation of core ideas...\n\n                Detailed Analysis\n                Techniques and methods for analysis...\n\n                Subsection: Specialized Techniques\n                Further details on specialized techniques...\n\n                Future Directions\n                Insights into upcoming trends...\n\n                Subsection: Next Steps in Research\n                Discussion of new areas of study...\n\n                Conclusion\n                Final remarks and summary.\n                ', max_num=6), Headlines(headlines=['Introduction', 'Main Concepts', 'Detailed Analysis', 'Subsection: Specializ

In [46]:
import tiktoken 

In [50]:
tokenizer = tiktoken.get_encoding("o200k_base")

In [58]:
text = kg.nodes[5].properties["page_content"]
tokens = tokenizer.encode(text)